# Conclusion
I was wrong. Generation of word pairs ignore the condition that the inner masks are symmetric. 

## todo
- relax punctuation

From https://czechtheworld.com/best-palindromes/#palindromes-symmetric-by-words

- You can cage a swallow, can’t you, but you can’t swallow a cage, can you?
- Fall leaves as soon as leaves fall.
- King, are you glad you are king?
- So patient a nurse to nurse a patient so.
- First ladies rule the state, and state the rule: “Ladies First!”
- Do I, like, look like I do?
- Sorry, I am very awkward. Very am I sorry.
- Is it crazy how saying sentences backwards creates backwards sentences saying how crazy it is?

In [1]:
from random import random
import numpy as np
import scipy
from transformers import AutoTokenizer, BertForMaskedLM
import torch

In [ ]:
uncasedTok = AutoTokenizer.from_pretrained("bert-base-uncased")
casedTok = AutoTokenizer.from_pretrained("bert-base-cased")
uncasedBert = BertForMaskedLM.from_pretrained("bert-base-uncased")
casedBert = BertForMaskedLM.from_pretrained("bert-base-cased")
CASED = 'CASED'
UNCASED = 'UNCASED'

In [18]:
def getMode(logits):
    return logits[0].argmax(dim=1)

In [20]:
def extend(casing, response_len, prompt, ends_with):
    if casing is CASED:
        bert = casedBert
        tok = casedTok
    else:
        bert = uncasedBert
        tok = uncasedTok
    input_idx = tok.encode(prompt.strip(' '))
    assert input_idx[-1] == tok.sep_token_id
    input_idx.pop(-1)
    len_input = len(input_idx)
    input_idx += (
        [tok.mask_token_id] * response_len + 
        [tok.convert_tokens_to_ids(ends_with)] + 
        [tok.sep_token_id]
    )
    print(tok.decode(input_idx))
    buffer = input_idx[:]
    
    for i in range(response_len):
        logits = bert(torch.tensor([buffer]))[0]
        prediction = getMode(logits)
        buffer[len_input + i] = prediction[len_input + i]
#         print(i, tok.decode(buffer))
    print()
    print(tok.decode(buffer))

In [21]:
extend(UNCASED, 16, 'Nobody expected the cancellation of the flight, so people began to panic. What if the representatives cannot make it in time?', '?')
print()
extend(UNCASED, 16, 'Nobody expected the cancellation of the flight, so people began to panic. What if the representatives cannot make it in time?', '.')
print()
extend(CASED, 16, 'Nobody expected the cancellation of the flight, so people began to panic. What if the representatives cannot make it in time?', '?')
print()
extend(CASED, 16, 'Nobody expected the cancellation of the flight, so people began to panic. What if the representatives cannot make it in time?', '.')

[CLS] nobody expected the cancellation of the flight, so people began to panic. what if the representatives cannot make it in time? [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK]? [SEP]

[CLS] nobody expected the cancellation of the flight, so people began to panic. what if the representatives cannot make it in time? how long is it going to take? and how long will it take to arrive? [SEP]

[CLS] nobody expected the cancellation of the flight, so people began to panic. what if the representatives cannot make it in time? [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK]. [SEP]

[CLS] nobody expected the cancellation of the flight, so people began to panic. what if the representatives cannot make it in time? " asked the american. " yes, " he replied, " it will not. [SEP]

[CLS] Nobody expected the cancellation of the flight, so people began to panic. What if the r

In [23]:
extend(UNCASED, 16, '', '?')

[CLS] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK]? [SEP]

[CLS] " what the hell are you doing, " he said, " getting a gun? [SEP]


In [27]:
extend(CASED, 16, 'In reality, my dog', '.')

[CLS] In reality, my dog [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK]. [SEP]

[CLS] In reality, my dog is the only - the - best - thing that could ever be in my life. [SEP]


In [ ]:
# def proposeReversal(prompt):
#     words = [*reversed(prompt.split(' '))]
#     return ' '.join(words)

In [62]:
def crab(response_len, prompt, ends_with, sampleMethod, casing, seed=None):
    if casing is CASED:
        bert = casedBert
        tok = casedTok
    else:
        bert = uncasedBert
        tok = uncasedTok
    input_idx = tok.encode(prompt.strip(' '))[1:-1]
    buffer = [None] * (
        1 + len(input_idx) + response_len + len(input_idx) + 1 + 1
    )
    acc = 0
    buffer[acc] = tok.cls_token_id
    acc += 1
    buffer[acc : acc + len(input_idx)] = input_idx
    acc += len(input_idx)
    buffer[acc : acc + response_len] = [tok.mask_token_id] * response_len
    acc += response_len
    buffer[acc : acc + len(input_idx)] = reversed(input_idx)
    acc += len(input_idx)
    if ends_with:
        buffer[acc] = tok.convert_tokens_to_ids(ends_with)
        acc += 1
    else:
        buffer.pop(-1)
    buffer[acc] = tok.sep_token_id
    acc += 1
    assert acc == len(buffer)
    
    print(tok.decode(buffer))
    for i in range((response_len + 1) // 2):
#         print(i, tok.decode(buffer))
        left  = 1 + len(input_idx) + i
        right = 1 + len(input_idx) + response_len - i - 1
        if seed is not None:
            pick = seed[i]
        else:
            pick = 0
        word_id = sampleMethod(buffer, left, right, pick, bert)
        buffer[left] = word_id
        buffer[right] = word_id
#     print()
    print(tok.decode(buffer))

In [29]:
def fastest(buffer, left, right, _, bert):
    logits = bert(torch.tensor([buffer]))[0].detach()
    logit = (
        scipy.special.softmax(logits[0][left]) * 
        scipy.special.softmax(logits[0][right])
    )
    return logit.argmax()

Optimization: stop looking when left prob < current best shot

In [92]:
def slowest(buffer, left, right, pick, bert):
    logits = bert(torch.tensor([buffer]))[0].detach()
    left_probs = scipy.special.softmax(logits[0][left])
    left_rank = np.argsort(left_probs)
    probs = []
    max_prob = [0]
    for i, word_id in enumerate(reversed(left_rank)):
#         print(i)
        left_prob = left_probs[word_id]
#         print(left_prob)
        if left_prob < max_prob[-1]:
            print('  Early stopping at', i, 'which is', i / len(left_probs))
            break
        buffer[left] = word_id
        logits = bert(torch.tensor([buffer]))[0].detach()
        right_probs = scipy.special.softmax(logits[0][right])
        prob = left_prob * right_probs[word_id]
#         print('*', right_probs[word_id], '=', prob)
        probs.append((word_id, prob))
        max_prob.append(prob)
        max_prob.sort(reverse = True)
        max_prob = max_prob[:pick + 1]
    s = sorted(probs, key=lambda x:x[1])
    return s[-1 - pick][0]

In [93]:
# TAIL_CUT = 0
# def slowestRand(buffer, left, right):
#     ...

In [94]:
crab(1, 'Fall leaves as', '.', slowest, UNCASED)
print()
crab(1, 'Fall leaves as', '.', slowest, CASED, [3])

[CLS] fall leaves as [MASK] as leaves fall. [SEP]
  Early stopping at 1 which is 3.2763252735731606e-05
[CLS] fall leaves as soon as leaves fall. [SEP]

[CLS] Fall leaves as [MASK] as leaves Fall. [SEP]
  Early stopping at 4 which is 0.00013795006207752792
[CLS] Fall leaves as easily as leaves Fall. [SEP]


In [95]:
crab(3, 'Fall leaves', '.', slowest, UNCASED, [1, 0])

[CLS] fall leaves [MASK] [MASK] [MASK] leaves fall. [SEP]
  Early stopping at 30 which is 0.000982897582071948
  Early stopping at 1 which is 3.2763252735731606e-05
[CLS] fall leaves falling. falling leaves fall. [SEP]


In [96]:
crab(5, 'Fall', '.', slowest, UNCASED, [5, 0, 0])

[CLS] fall [MASK] [MASK] [MASK] [MASK] [MASK] fall. [SEP]
  Early stopping at 20 which is 0.0006552650547146321
  Early stopping at 1 which is 3.2763252735731606e-05
  Early stopping at 1 which is 3.2763252735731606e-05
[CLS] fall = fall. fall = fall. [SEP]


In [97]:
crab(1, 'So patient a nurse', '.', slowest, UNCASED, [5])

[CLS] so patient a nurse [MASK] nurse a patient so. [SEP]
  Early stopping at 6 which is 0.00019657951641438963
[CLS] so patient a nurse to nurse a patient so. [SEP]


In [98]:
crab(3, 'So patient a', '.', slowest, UNCASED, [2, 0])

[CLS] so patient a [MASK] [MASK] [MASK] a patient so. [SEP]
  Early stopping at 25 which is 0.0008190813183932901
  Early stopping at 1 which is 3.2763252735731606e-05
[CLS] so patient a doctor a doctor a patient so. [SEP]


In [99]:
crab(
    1, 'Is it crazy how saying sentences backwards', '?', slowest, 
    UNCASED, [9], 
)

[CLS] is it crazy how saying sentences backwards [MASK] backwards sentences saying how crazy it is? [SEP]
  Early stopping at 10 which is 0.00032763252735731603
[CLS] is it crazy how saying sentences backwards makes backwards sentences saying how crazy it is? [SEP]


In [100]:
# crab(
#     3, 'Is it crazy how saying sentences', '?', slowest, 
#     UNCASED, [0, 9-9], 
# )